This script use to train DNN and CNN model on all splited train data

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/ESE529-project") 
os.listdir()

['final_train_data.RData',
 'train_price.RData',
 'true_price.RData',
 '__pycache__',
 'utils.py',
 'DNN (1).py',
 'dnn_model1.h5',
 'train.tsv',
 'combine_train_data.RData',
 'group_price.RData',
 'classification_model.ipynb',
 'test_index.Rdata',
 'run_keras_model.ipynb',
 'train_index.Rdata',
 'test_index.RData',
 'CNN_prediction.txt',
 'CNN_test.h5',
 'DNN_prediction.txt',
 'final_DNN_CNN.ipynb']

In [3]:

import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [0]:
robjects.r.load('final_train_data.RData')
final_train_data = robjects.r['final_train_data']
final_train_data=np.array(final_train_data)

In [0]:
robjects.r.load('true_price.RData')
true_price = robjects.r['true_price']
true_price=np.array(true_price)

In [0]:
robjects.r.load('train_index.Rdata')
train_index = robjects.r['v']
train_index=np.array(train_index)

In [0]:
robjects.r.load('test_index.RData')
test_index = robjects.r['test_index']
test_index=np.array(test_index)

In [0]:
test_index.shape

(444499,)

In [0]:
train_price=np.log(true_price+1)

In [10]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 123

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

Using TensorFlow backend.


In [0]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input,Dense,BatchNormalization,Activation,Conv1D,\
    Flatten,Reshape,Dropout,concatenate,MaxPooling1D,add
from tensorflow.keras import Model
from tensorflow.keras import regularizers

def DNN(input_dim,layer_list,activation="relu",dropout=0.2):
    input=Input((input_dim,))
    for i in range(len(layer_list)):
        if i==0:
            dense=Dense(layer_list[i],activation=activation,
                        kernel_regularizer=regularizers.l2(0.001))(input)
            dense=BatchNormalization()(dense)
            dense=Activation(activation)(dense)
            if dropout>0:
                dense=Dropout(dropout)(dense)
        else:
            dense=Dense(layer_list[i],activation=activation,
                        kernel_regularizer=regularizers.l2(0.001))(dense)
            dense=BatchNormalization()(dense)
            dense=Activation(activation)(dense)
            if dropout>0:
                dense=Dropout(dropout)(dense)
    output=Dense(1)(dense)
    DNN_model=Model(input,output)
    return DNN_model


In [0]:
def inception(filter_size):

    def f(input_layer):
        inception1=Conv1D(filter_size,1,padding="same",activation="relu")(input_layer)

        inception2=Conv1D(filter_size,1,padding="same",activation="relu")(input_layer)
        inception2=Conv1D(filter_size,3,padding="same",activation="relu")(inception2)

        inception3=Conv1D(filter_size,1,padding="same",activation="relu")(input_layer)
        inception3=Conv1D(filter_size,3,padding="same",activation="relu")(inception3)
        inception3=Conv1D(filter_size,3,padding="same",activation="relu")(inception3)

        inception4=MaxPooling1D(padding="same",strides=1)(input_layer)
        inception4=Conv1D(filter_size,3,padding="same",activation="relu")(inception4)
        concat=concatenate([inception1,inception2,inception3,inception4],axis=-1)
        concat=Conv1D(filter_size,1,padding="same",activation="relu")(concat)

        output_layer=add([input_layer,concat])
        return output_layer
    return f

def CNN(input_dim1,input_dim2,filter_list1,dropout=0.2,reg=0.0001):
    catgorical_input=Input((input_dim1,))
    catgorical_dense=Dense(256,activation="relu")(catgorical_input)
    catgorical_dense=Dense(512,activation="relu")(catgorical_dense)

    text_input=Input((input_dim2,))
    text_reshape=Reshape((input_dim2,1))(text_input)
    for i in range(len(filter_list1)):
        if i == 0:

            text_conv = Conv1D(filter_list1[i], 5, padding="same", activation="relu",kernel_regularizer=regularizers.l2(reg))(text_reshape)
        else:
            text_conv = Conv1D(filter_list1[i], 5, padding="same", activation="relu",kernel_regularizer=regularizers.l2(reg))(text_conv)
        text_conv = inception(filter_list1[i])(text_conv)
        text_conv = BatchNormalization()(text_conv)
        text_conv = Activation("relu")(text_conv)
        text_conv = Dropout(dropout)(text_conv)

    text_flatten=Flatten()(text_conv)
    print(K.int_shape(text_flatten))
    concat=concatenate([catgorical_dense,text_flatten])

    final_dense=Dense(2048,activation="relu",kernel_regularizer=regularizers.l2(reg))(concat)
    final_dense=Dense(1024,activation="relu",kernel_regularizer=regularizers.l2(reg))(final_dense)
    final_dense=Dense(256,activation="relu",kernel_regularizer=regularizers.l2(reg))(final_dense)
    final_dense=Dense(64,activation="relu",kernel_regularizer=regularizers.l2(reg))(final_dense)
    output=Dense(1)(final_dense)

    CNN_model=Model([catgorical_input,text_input],output)
    return CNN_model


In [0]:
def mse(y_pre,y_tr):
    original_y_pre=np.exp(y_pre)-1
    num_sample=y_pre.shape[0]
    mse_result=np.sum(np.square(original_y_pre-y_tr))/num_sample
    return mse_result

def rmsle(y_pre,y_tr):
    nat_y_tr=np.log(y_tr+1)
    num_sample=y_pre.shape[0]
    rmsle_result=np.sqrt(np.sum(np.square(y_pre-nat_y_tr))/num_sample)
    return rmsle_result

In [0]:
train_index=train_index-1

In [0]:
test_index=test_index-1

In [0]:
train_y=train_price[train_index]
test_y=train_price[test_index]

In [0]:
final_train_x=final_train_data[train_index,:]
final_test_x=final_train_data[test_index,:]

In [0]:
dnn_model=DNN(188,[512,1024,2048,512,256,64],dropout=0.2)
dnn_model.compile(loss="mse", 
                optimizer='adam',
                metrics=['mse'])
dnn_model.fit(final_train_x,train_y,batch_size=2048,epochs=100,validation_data=(final_test_x,test_y))

Epoch 1/100
507/507 [==============================] - 24s 48ms/step - loss: 5.2758 - mse: 2.0555 - val_loss: 3.0667 - val_mse: 0.5150
Epoch 2/100
507/507 [==============================] - 24s 47ms/step - loss: 2.6005 - mse: 0.7133 - val_loss: 1.7560 - val_mse: 0.4522
Epoch 3/100
507/507 [==============================] - 24s 47ms/step - loss: 1.4605 - mse: 0.5510 - val_loss: 1.0344 - val_mse: 0.4373
Epoch 4/100
507/507 [==============================] - 24s 47ms/step - loss: 0.9050 - mse: 0.4903 - val_loss: 0.6970 - val_mse: 0.4207
Epoch 5/100
507/507 [==============================] - 24s 47ms/step - loss: 0.6523 - mse: 0.4522 - val_loss: 0.5203 - val_mse: 0.3786
Epoch 6/100
507/507 [==============================] - 24s 47ms/step - loss: 0.5333 - mse: 0.4243 - val_loss: 0.4586 - val_mse: 0.3741
Epoch 7/100
507/507 [==============================] - 24s 47ms/step - loss: 0.4755 - mse: 0.4068 - val_loss: 0.4056 - val_mse: 0.3489
Epoch 8/100
507/507 [==============================] - 

In [0]:
test_prediction=dnn_model.predict(final_test_x)

In [0]:
np.savetxt("DNN_prediction.txt",test_prediction)

In [0]:

cnn_model=CNN(132,56,[8,32,64])
cnn_model.compile(loss='mse', 
                optimizer='adam',
                metrics=['mse'])
cnn_model.fit([final_train_x[:,:132],final_train_x[:,132:]],train_y,batch_size=2048,epochs=100,validation_data=([final_test_x[:,:132],final_test_x[:,132:]],test_y))


(None, 3584)
Epoch 1/100
507/507 [==============================] - 146s 287ms/step - loss: 1.7250 - mse: 1.2748 - val_loss: 0.8055 - val_mse: 0.3986
Epoch 2/100
507/507 [==============================] - 144s 284ms/step - loss: 0.7534 - mse: 0.3845 - val_loss: 0.7089 - val_mse: 0.3744
Epoch 3/100
507/507 [==============================] - 144s 284ms/step - loss: 0.6755 - mse: 0.3705 - val_loss: 0.6335 - val_mse: 0.3554
Epoch 4/100
507/507 [==============================] - 144s 284ms/step - loss: 0.6162 - mse: 0.3614 - val_loss: 0.5840 - val_mse: 0.3506
Epoch 5/100
507/507 [==============================] - 144s 284ms/step - loss: 0.5692 - mse: 0.3544 - val_loss: 0.5336 - val_mse: 0.3361
Epoch 6/100
507/507 [==============================] - 144s 284ms/step - loss: 0.5295 - mse: 0.3471 - val_loss: 0.4994 - val_mse: 0.3315
Epoch 7/100
507/507 [==============================] - 144s 284ms/step - loss: 0.4975 - mse: 0.3423 - val_loss: 0.4738 - val_mse: 0.3307
Epoch 8/100
507/507 [=======

In [0]:
test_prediction=cnn_model.predict([final_test_x[:,:132],final_test_x[:,132:]])

In [0]:
np.savetxt("CNN_prediction.txt",test_prediction)

In [17]:
#run cnn model on full train data to get a final CNN model
cnn_model=CNN(132,56,[8,32,64])
cnn_model.compile(loss='mse', 
                optimizer='adam',
                metrics=['mse'])
cnn_model.fit([final_train_data[:,:132],final_train_data[:,132:]],train_price,batch_size=2048,epochs=100)


(None, 3584)
Epoch 1/100
724/724 [==============================] - 55s 75ms/step - loss: 1.3925 - mse: 0.9726
Epoch 2/100
724/724 [==============================] - 54s 75ms/step - loss: 0.6782 - mse: 0.3757
Epoch 3/100
724/724 [==============================] - 54s 75ms/step - loss: 0.5913 - mse: 0.3609
Epoch 4/100
724/724 [==============================] - 54s 75ms/step - loss: 0.5310 - mse: 0.3501
Epoch 5/100
724/724 [==============================] - 54s 75ms/step - loss: 0.4872 - mse: 0.3432
Epoch 6/100
724/724 [==============================] - 54s 75ms/step - loss: 0.4524 - mse: 0.3371
Epoch 7/100
724/724 [==============================] - 54s 75ms/step - loss: 0.4232 - mse: 0.3312
Epoch 8/100
724/724 [==============================] - 54s 75ms/step - loss: 0.4000 - mse: 0.3270
Epoch 9/100
724/724 [==============================] - 54s 75ms/step - loss: 0.3793 - mse: 0.3217
Epoch 10/100
724/724 [==============================] - 54s 75ms/step - loss: 0.3650 - mse: 0.3196
Epoch 

In [0]:
cnn_model.save("final_cnn.h5")